In [2]:
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [3]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 이성용이야.")],
    config=config,
)

print(response.content)

안녕하세요, 이성용님! 어떻게 도와드릴까요?


In [4]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 이성용입니다. 다른 질문이나 이야기가 있으신가요?


In [5]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 당신의 이름을 알지 못합니다. 어떤 이름으로 불리고 싶으신가요?'

In [7]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config
)

response.content

'우리는 당신의 이름이 이성용이라는 것에 대해 이야기했어요. 그 외에 더 궁금한 점이나 이야기하고 싶은 내용이 있으신가요?'

In [9]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content="내가 어느 나라 사람인지 맞춰보고, 그 나라의 국가를 불러줘.")],
    config=config,
):
    print(r.content, end="|")

|이|성|용|이라는| 이름|은| 주|로| 한국|에서| 많이| 사용|되|므로|,| 제가| 추|측|하기|로|는| 한국| 분|이|실| 가능|성이| 높|습니다|.| 한국|의| 국가|인| "|애|국|가|"|를| 불|러|드|리고| 싶|습니다|.| 

|애|국|가는| 아래|와| 같은| 내용|으로| 시작|합니다|:

|**|"|동|해| 물|과| 백|두|산|이| 마|르고| 닳|도록|..."|**

|제가| 맞|췄|나요|?| 그렇|지| 않|다면|,| 어떤| 나라| 사람|인지| 말씀|해|주|시면| 그| 나라|의| 국가|에| 대해| 알려|드|릴|게|요|!||||